### Imports

In [ ]:
from pycelonis import get_celonis
from pycelonis import pql

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import xgboost as xgb

### Load Data

#### From DM (recommended)

In [ ]:
c = get_celonis()

# Set DM

dm_id = 'input_data_dm_id'
dm = c.datamodels.find(dm_id)
dm

# Set PQL Query of the data to load

query = pql.PQL()

cols_to_load = [("table_1.col_name_1", "col_pretty_name_1")
            ,("table_2.col_name_2", "col_pretty_name_2")
               ]

for col,pretty_col in cols_to_load:
    query += pql.PQLColumn(col,pretty_col)

query += pql.PQLFilter("FILTER TBD;")

loaded_df = dm._get_data_frame(query)

loaded_df.head()

#### From Analysis (if needed)

In [ ]:
c = get_celonis()

workspace_id = "TBD"
workspace = c.workspaces.find(workspace_id)

analysis_name = 'TBD'
analysis = workspace.analyses.find(analysis_name, False)
component = analysis.draft.components.find("TBD")
loaded_df = component.get_data_frame()

loaded_df.head()

##### Study

In [ ]:
loaded_df.shape

# Feature distribution
col_name = 'TBD'
print('For col ',col_name,' # NaN values are ',loaded_df[col_name].isna().shape[0])
print('Value Distribution is ',loaded_df[col_name].value_counts().sort_values(ascending=False))

# Target Distribution
target_col = 'TBD'
print(loaded_df[target_col].isna().shape[0])
print(loaded_df[target_col].value_counts().sort_values(ascending=False))
100.0*loaded_df[target_col].mean()

# Plot Feature vs Target
plt.scatter(loaded_df[col_name],loaded_df[target_col])

### Preprocessing

In [ ]:
# Create copy of loaded df

input_df = loaded_df.copy()

# Filter rows

input_df = input_df[input_df['col_to_filter'] == 'condition to match']
input_df = input_df[input_df['date_col_to_filter'] >= dt.date(2021,2,10)] # Date filter
input_df = input_df[input_df['col_to_filter'] >= 'min condition']
input_df = input_df[input_df['col_to_filter'] <= 'max condition']
input_df = input_df[input_df['col_to_filter'].str.contains('string pattern to filter on')] # Filter based on column contains a certain string pattern

# Clean values

input_df = input_df.dropna() # Remove rows with some empty values
input_df = input_df[~input_df['col'].isna()] # Remove rows with empty value in a certain column

input_df.loc[input_df['column'].isna()] = 'value_for_na' # Fill empty values of a certain column with a value

# Change data types

input_df = input_df['column'].astype(int) # Cast column as certain type, such as Int
input_df = input_df['date_column'].dt.date() # Cast date column as Date type

# Sort rows

input_df = input_df.sort_values(by=['column_to_sort_on'],ascending=True)

# Set row ID as index

input_df['CASE_KEY'] = input_df['pk1'] + input_df['pk2'] + input_df['pk3']
input_df = input_df.set_index('CASE_KEY')

input_df.head()

### Feature Engineering

In [ ]:
## Select columns for the model

feature_cols = ['col1'
                ,'col2']
input_df = input_df[feature_cols]

## Transform existing columns

input_df['column'] = input_df['column']**2 # Square existing column
input_df['column'] = np.sqrt(input_df['column']) # SQRT existing column

## Create new columns

input_df['new_column'] = input_df['column_1'] + input_df['column_2'] # New column with columns 1 and 2
input_df['new_column'] = np.where(input_df['column_1'] == 'condition','Value_if_true','Value_if_false') # Create flag based on a column

input_df['new_column'] = input_df.groupby('column_to_group_by')['column_to_aggregate'].transform('mean') # Aggregate column, such as Average or Sum
input_df['new_column'] = input_df.groupby('column_to_group_by')['column_to_aggregate'].rolling(7).mean().reset_index(0,drop=True) # Rolling average
input_df['new_shifted_column'] = input_df.groupby('column_to_group_by')['column_to_shift'].shift(-1) # Shift values

## Encode categorical columns

input_df = pd.get_dummies(input_df,columns=['col1'],drop_first=True)

input_df.head()

### Train/Test/Forecast Split

In [ ]:
# Set Target=y column

target_col = 'target_col_name'

# Forecast mask

forecast_msk = input_df[target_col].isna()

# X/y split

train_test_df = input_df[~forecast_msk]

X_df = train_test_df.drop(columns=[target_col])
y_df = train_test_df[target_col]


## Train/Test split OPTION 1: Set split w/ mask

msk = ~forecast_msk&(input_df['date_column']<dt.date(2021,2,10)) # Date split

msk = (input_df['value_column']<='value_for_split') # Value split

X_train, X_test, y_train, y_test = X_df[msk],X_df[~msk],y_df[msk],y_df[~msk]


## Train/Test split OPTION 2: Random split w/ Scikit-learn

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

### Train Model (choose between Regression and Classification)

#### Regression model (predict a value i.e. # Sales Orders per day)

In [ ]:
## OPTION REGRESSION - Set the XGBoost model
# Doc here: https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
xgb_model = xgb_regression = xgb.XGBRegressor(objective='reg:squarederror'
                                  
                                  # IMPORTANT - max depth of trees
                                  ,max_depth=5
                                  
                                   # IMPORTANT - learning speed of the algoritm, the smaller the more precise but slower e.g. needs higher n_estimators
                                  ,learning_rate=0.1
                                   
                                  # % columns used at each tree split, can be defaulted to 1
                                  ,colsamply_bytree=0.8
                                
                                  # Regularization factor, to avoid overfitting
                                  ,alpha=10
                                  
                                  # IMPORTANT - # trees
                                  ,n_estimators=30
                                 )

## Fit model to train set

# standard Option: fit train
xgb_model.fit(X_train,y_train,verbose=True)

# better Option: fit train and validate on test
eval_set = [(X_train, y_train),(X_test, y_test)]
xgb_model.fit(X_train, y_train, eval_metric=["rmse"], eval_set=eval_set, verbose=True)

xgg_model

#### Classification model (predict a category i.e. 'Late' or 'On time')

In [ ]:
# OPTION CLASSIFICATION - Set the XGBoost model
# Doc here: https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
xgb_model = xgb_classifier = xgb.XGBClassifier(objective='binary:logistic'
                                  
                                  # IMPORTANT - max depth of trees
                                  ,max_depth=8
                                  
                                   # IMPORTANT - learning speed of the algoritm, the smaller the more precise but slower e.g. needs higher n_estimators
                                  ,learning_rate=0.1
                                   
                                  # % columns used at each tree split, can be defaulted to 1
                                  ,colsample_bytree=1.0
                                
                                  # Regularization factor, to avoid overfitting
                                  #,reg_lambda=10
                                  
                                  # IMPORTANT - # trees
                                  ,n_estimators=30
                                  
                                  # % Subsample of the dataset to consider for Train (if Train set is too large)
                                  ,subsample=1.0
                                 )

## Train model on Train set

# standard Option: fit train
xgb_model.fit(X_train,y_train,verbose=True)

# better Option: fit train and validate on test
eval_set = [(X_train, y_train),(X_test, y_test)]
xgb_model.fit(X_train, y_train, eval_metric=["logloss"], eval_set=eval_set, verbose=True)

xgb_model

#### Save model (Optional)

In [ ]:
# Export Results to csv
from datetime import date
today = date.today()
xgb_model.save_model('xgb_model_'+str(today)+'.model')

### Test Model

#### Run model on Test set

In [ ]:
print('X_test has shape... ',X_test.shape)
print('Should have as many columns as X_train with shape... ',X_train.shape)

## Predict y for Test set
pred_y_test = xgb_model.predict(X_test)
# Turn into Df
pred_test_df = pd.DataFrame(pred_y_test,index=X_test.index,columns=['PREDICTION'])

## (Optional) For binary Classification: Predict probabilities too
proba_pred_y_test = xgb_model.predict_proba(X_test)[:,1]
# Turn into Df
proba_pred_test_df = pd.DataFrame(proba_pred_y_test,index=X_test.index,columns=['PROBA_PREDICTION'])


## Add columns to Test Predictions Df (such as Case Key or attributes like Country, Material type etc)

def add_columns_to_results(results_df,cols_df,col_names):
    if col_names is None:
        enriched_results = results_df.join(cols_df,how='left')
    else:
        enriched_results = results_df.join(cols_df[col_names],how='left')
    return enriched_results

# Add Proba Preds y to df
pred_test_df = add_columns_to_results(pred_test_df,proba_pred_test_df,None)

# Add True y to df
pred_test_df = add_columns_to_results(pred_test_df,y_test,None)

# Add input columns to df
cols_to_add = list(input_df.columns)
cols_to_add.remove(target_col)
pred_test_df = add_columns_to_results(pred_test_df,input_df,cols_to_add)

## Print Test Predictions Df
pred_test_df.head(10)

#### Evaluate Model on Test set

In [ ]:
## Option REGRESSION

# TBD

In [ ]:
## Option CLASSIFICATION

# Feature importance

xgb.plot_importance(xgb_model)

# Confusion Matrix

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_test, pred_y_test).ravel()
print('Confusion matrix ',confusion_matrix(y_test, pred_y_test))

# Precision = % correct positives = True Positives / (True Positives + False Positives)

from sklearn.metrics import precision_score

precision = precision_score(y_test, pred_y_test, average='binary')
print('Precision: ',precision)

# Recall = % detected positives = TruePositives / (TruePositives + FalseNegatives)

from sklearn.metrics import recall_score

recall = recall_score(y_test, pred_y_test, average='binary')
print('Recall: ',recall)

### Forecast Model (Optional, for out-of-sample Predictions)

In [ ]:
## Predict y for Forecast set
pred_y_forecast = xgb_model.predict(X_forecast)

pred_forecast_df = pd.DataFrame(pred_y_forecast,index=X_forecast,columns=['PREDICTION'])

## Combine Forecasts and Test predictions in same Df

# Add input columns to Forecast Preds Df
print('Adding same input columns as for Test Df: ',cols_to_add)
pred_forecast_df = add_columns_to_results(pred_forecast_df,input_df,cols_to_add)
# Add Subset column for Test vs Forecast
pred_test_df['Subset'] = 'Test'
pred_forecast_df['Subset'] = 'Forecast'

# Combine in same Df
all_preds_df = pd.concat([pred_test_df, pred_forecast_df])

# Print Test & Forecast Df
all_preds_df.head(5)

#### Export Results

##### Data Engineering

In [ ]:
# Copy Predictions df for Export
export_preds_df = all_preds_df.copy()

# Rename columns (to match current DM tables field names if it helps)
cols_to_rename = {'col_1':'new_col_1','col_2':'new_col_2'}
export_preds_df = export_preds_df.rename(columns=cols_to_rename)

# Add other columns (for Joining purposes or Analysis drilldowns)

cols_to_add_to_export = ['col_1','col_2']
export_preds_df = add_columns_to_results(export_preds_df,loaded_df,cols_to_add_to_export)

export_preds_df.head(5)

##### Push Results to DM

In [ ]:
# Verify export Predictions one last time before push to DM
print('Shape of the Export is ',export_preds_df.shape)

print('Test and Forecast sets sizes are ',export_preds_df['Subset'].value_counts())

print('Test predictions preview is ',export_preds_df[export_preds_df['Subset']=='Test'].head())
print('Forecast predictions preview is ',export_preds_df[export_preds_df['Subset']=='Forecast'].head())

In [ ]:
# Set DM where Predictions export should be pushed to
export_dm_id = 'export_DM_ID'
export_dm = c.datamodels.find(export_dm_id)
export_dm

# Push Predictions to DM
table_name = '_ML_PREDICTIONS'
export_dm.push_table(exports_preds_df,,if_exists='replace',reload_datamodel = False)

#### Push Results to DM, with Historical table

In [ ]:
history_table_name = f"_ML_PREDICTIONS_HISTORY"
try:
    dm.pool.append_table(
        df_or_path=exports_preds_df,
        table_name=history_table_name,
    )
except ValueError: # Table does not exist
    print(f"Create table {history_table_name}")
    dm.pool.create_table(
        df_or_path=exports_preds_df,
        table_name=history_table_name,
        if_exists="error"
    )

    print("Add table to datamodel...")
    dm.add_table_from_pool(history_table_name)
    source_table = 'TABLE_TO_JOIN_TO'
    target_table = history_table_name
    dm.create_foreign_key(source_table,
                          target_table,
                          [(col.name, col.name) for col in ['join_col1','join_col2']])
